In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('celeb_name_list.csv')        # 연예인 리스트
article_file = open('/Users/hayeong/DataScience/Data-Science/DSlocal/test_data.txt', 'r')      # 크롤링한 기사 제목들의 텍스트 파일 (구분자는 '\n'으로 가정)

# name_articles_dict: {name: <연예인 이름 리스트>, article: <기사 제목 리스트>}
articles_dict = {'Name':[], 'Article':[]}

df.head(10)

In [ ]:
while True:
    # 크롤링한 기사 제목 한 줄을 읽어옴
    article = article_file.readline()
    # 크롤링한 기사 제목을 모두 읽었으면 break
    if article == '':
        break
    article = article[:article.rfind(' ')]       # 기사 제목 뒤의 신문사 이름 제외시키기
    # namelist 안의 모든 연예인 이름에 대해서 기사 제목에 포함되는 이름이 있는 지 확인
    for name in df['Name']:
        # 보통 연예인 이름 뒤에 컴마가 오길래 '<이름>,'를 찾는걸로 해봄
        if name+',' in article:
            if not (name in articles_dict['Name']):
                articles_dict['Name'].append(name)
                articles_dict['Article'].append(article+'\n')
            else:
                index = articles_dict['Name'].index(name)
                if not (article in articles_dict['Article'][index]):
                    articles_dict['Article'][index] += article+'\n'
                
        # 제목에 포함된 연예인을 찾았어도 연예인 리스트 전체 탐색함
        # => 기사에 두 명 이상의 연예인이 있을 경우 모두 찾음
article_df = pd.DataFrame(articles_dict)

In [ ]:
article_df.head(10)

In [ ]:
# count_dict: {Name : 연예인 이름 리스트, Count : 언급 횟수}
count_dict = {'Name': articles_dict['Name'], 'Count': []}
i = 0
for name in articles_dict['Name']:
    count_dict['Count'].append(articles_dict['Article'][i].count('\n'))
    i +=1

count_df = pd.DataFrame(count_dict)
count_df.head()

In [ ]:
# df에 언급 횟수인 Count 컬럼 추가
merged_df = pd.merge(df, count_df, on='Name', how = 'outer')
merged_df.head()

In [ ]:
# 언급 되지 않은 연예인이 있는 행은 삭제
temp_df = merged_df.dropna(axis = 0, how = 'any')
# 언급 횟수 기준 내림차순으로 정렬
df = temp_df.sort_values(by = 'Count', ascending=False,)
df.head(20)

In [ ]:
### 생략 가능 ###
# 동명이인일 시 두 명 다 같은 언급횟수를 가지고 있음  ex)김희정(49세), 김희정(28세): 기사 하나에 언급되면 둘 다 count = 1이 됨
# duplicated: 동명이인인 이름의 리스트
duplicated = df[df.Name.duplicated()]['Name'][:20].tolist()
print('=== 동명이인인 이름 리스트 ===')
print(duplicated)


# 동명이인이 들어간 기사 체크하기
for key in duplicated:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

# 동명이인의 정보 확인하기
duplicated_df = df[df['Name'].isin(duplicated)]
duplicated_df.head()

In [ ]:
candidates = df['Name'].tolist()[:20]
print('=== Top 20 ===')
print(candidates)

In [ ]:
# 상위 20명의 기사가 제대로 된 것인지 체크하기
print('=== Check articles ===')
for key in candidates:
    print('>> '+key+'\n' + articles_dict['Article'][articles_dict['Name'].index(key)])

In [ ]:
merged_df = pd.merge(df, article_df, on='Name', how = 'outer')
# csv로 저장
merged_df.to_csv('candidates.csv', index=False, mode='w', encoding='utf-8-sig')
merged_df.head(20)